In [1]:
import numpy as np 
import re 
import os 
import pandas as pd
from nltk.tokenize import RegexpTokenizer , sent_tokenize
from urllib.request import urlopen
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import requests
import urllib.request,sys,time ,requests

In [2]:
stopWordsFile =     "C:\\Users\\kajol\\Desktop\\blackcoffer\\StopWords_Generic.txt"
positiveWordsFile = "C:\\Users\\kajol\\Desktop\\blackcoffer\\positive-words.txt"
nagitiveWordsFile = "C:\\Users\\kajol\\Desktop\\blackcoffer\\negative-words.txt"

In [3]:
input = pd.read_excel("C:\\Users\\kajol\\Desktop\\blackcoffer\\Input.xlsx")
input

def get_article_names(urls):
  titles = []
  for i in range (len(urls)):
    title = urls[i]
    title_clean = title[title.index( "m/" ) + 2 :-1]. replace('-' , ' ')
    titles.append(title_clean)
  return titles

urls =input["URL"]
urlsTitleDF = get_article_names(urls)
urlsTitleDF

['rising it cities and its impact on the economy environment infrastructure and city life by the year 2040 2',
 'rising it cities and their impact on the economy environment infrastructure and city life in future',
 'internet demands evolution communication impact and 2035s alternative pathways',
 'rise of cybercrime and its effect in upcoming future',
 'ott platform and its impact on the entertainment industry in future',
 'the rise of the ott platform and its impact on the entertainment industry by 2040',
 'rise of cyber crime and its effects',
 'rise of internet demand and its impact on communications and alternatives by the year 2035 2',
 'rise of cybercrime and its effect by the year 2040 2',
 'rise of cybercrime and its effect by the year 2040',
 'rise of internet demand and its impact on communications and alternatives by the year 2035',
 'rise of telemedicine and its impact on livelihood by 2040 3 2',
 'rise of e health and its impact on humans by the year 2030',
 'rise of e he

In [4]:
url = "https://insights.blackcoffer.com/how-people-diverted-to-telehealth-services-and-telemedicine"

page=requests.get(url , headers={"User-Agent": "XY"})  
soup = BeautifulSoup(page.text , 'html.parser')
#get title
title = soup . find("h1",attrs = { 'class' : 'entry-title'}).get_text()

#get article text
text = soup . find(attrs = { 'class' : 'td-post-content'}).get_text()
# break into lines and remove leading and trailing space on each
lines = (line.strip() for line in text.splitlines())
# break multi-headlines into a line each
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
# drop blank lines
text = '\n'.join(chunk for chunk in chunks if chunk)

In [5]:
# Loading positive words
with open(positiveWordsFile,'r') as posfile:
    positivewords=posfile.read().lower()
positiveWordList=positivewords.split('\n')


# Loading negative words
with open(nagitiveWordsFile ,'r' ,  encoding="ISO-8859-1") as negfile:
    negativeword=negfile.read().lower()
negativeWordList=negativeword.split('\n')

#Loading stop words dictionary for removing stop words

with open(stopWordsFile ,'r') as stop_words:
    stopWords = stop_words.read().lower()
stopWordList = stopWords.split('\n')
stopWordList[-1:] = []



display( positiveWordList[:6]  , negativeWordList[:6] , stopWordList[:6])
     

['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable']

['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably']

['about', 'above', 'after', 'again', 'all', 'am']

In [6]:
#tokenizeing module and filtering tokens using stop words list, removing punctuations
def tokenizer(text):
    text = text.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    filtered_words = list(filter(lambda token: token not in stopWordList, tokens))
    return filtered_words

def positive_score (text):
  posword=0
  tokenphrase = tokenizer(text)
  for word in tokenphrase :
    if word in positiveWordList:
       posword+=1
      
    retpos = posword
    return retpos 

def negative_score (text):
  negword=0
  tokenphrase = tokenizer(text)
  for word in tokenphrase :
    if word in negativeWordList : negword +=1

    retneg = negword 
    return retneg

def polarity_score (positive_score , negative_score) :
  return (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
#################################################
def total_word_count(text):
    tokens = tokenizer(text)
    return len(tokens)
#############################################
def AverageSentenceLenght (text):
  Wordcount = len(tokenizer (text))
  SentenceCount = len (sent_tokenize(text))
  if SentenceCount > 0 : Average_Sentence_Lenght = Wordcount / SentenceCount

  avg = Average_Sentence_Lenght

  return round(avg)


# Counting complex words
def complex_word_count(text):
    tokens = tokenizer(text)
    complexWord = 0
    
    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):
            pass
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                    vowels += 1
            if(vowels > 2):
                complexWord += 1
    return complexWord
def percentage_complex_word(text):
    tokens = tokenizer(text)
    complexWord = 0
    complex_word_percentage = 0
    
    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):
            pass
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                    vowels += 1
            if(vowels > 2):
                complexWord += 1
    if len(tokens) != 0:
        complex_word_percentage = complexWord/len(tokens)
    
    return complex_word_percentage

def fog_index(averageSentenceLength, percentageComplexWord):
    fogIndex = 0.4 * (averageSentenceLength + percentageComplexWord)
    return fogIndex

In [7]:
URLS = input ["URL"]
URLS

0     https://insights.blackcoffer.com/rising-it-cit...
1     https://insights.blackcoffer.com/rising-it-cit...
2     https://insights.blackcoffer.com/internet-dema...
3     https://insights.blackcoffer.com/rise-of-cyber...
4     https://insights.blackcoffer.com/ott-platform-...
                            ...                        
95    https://insights.blackcoffer.com/what-is-the-r...
96    https://insights.blackcoffer.com/impact-of-cov...
97    https://insights.blackcoffer.com/contribution-...
98    https://insights.blackcoffer.com/how-covid-19-...
99    https://insights.blackcoffer.com/how-will-covi...
Name: URL, Length: 100, dtype: object

In [8]:
corps =[]
for URL in URLS:

  page=requests.get(url , headers={"User-Agent": "XY"})  
  soup = BeautifulSoup(page.text , 'html.parser')
  #get title
  title = soup . find("h1",attrs = { 'class' : 'entry-title'}).get_text()

  #get article text
  text = soup . find(attrs = { 'class' : 'td-post-content'}).get_text()
  # break into lines and remove leading and trailing space on each
  lines = (line.strip() for line in text.splitlines())
  # break multi-headlines into a line each
  chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
  # drop blank lines
  text = '\n'.join(chunk for chunk in chunks if chunk)
  corps.append(text)


In [10]:
#df = pd.DataFrame(corps,urlsTitleDF)
df = pd.DataFrame({'title':urlsTitleDF,'corps': corps})
df["total word count"] = df["corps"] . apply (total_word_count)
df["percentage_complex_word"] = df["corps"] . apply (percentage_complex_word)
df["complex_word_count"] = df["corps"] . apply (complex_word_count)
df["AverageSentenceLenght"] = df["corps"] . apply (AverageSentenceLenght)
df["positive_score"] = df["corps"] . apply (positive_score)
df["negative_score"] = df["corps"] . apply (negative_score)
df["polarity_score"] = np.vectorize(polarity_score)(df['positive_score'],df['negative_score'])

df

,title,corps,total word count,percentage_complex_word,complex_word_count,AverageSentenceLenght,positive_score,negative_score,polarity_score
0,rising it cities and its impact on the economy...,The future of telehealth stays positive\nTo as...,1141,0.449606,513,14,0,0,0.0
1,rising it cities and their impact on the econo...,The future of telehealth stays positive\nTo as...,1141,0.449606,513,14,0,0,0.0
2,internet demands evolution communication impac...,The future of telehealth stays positive\nTo as...,1141,0.449606,513,14,0,0,0.0
3,rise of cybercrime and its effect in upcoming ...,The future of telehealth stays positive\nTo as...,1141,0.449606,513,14,0,0,0.0
4,ott platform and its impact on the entertainme...,The future of telehealth stays positive\nTo as...,1141,0.449606,513,14,0,0,0.0
...,...,...,...,...,...,...,...,...,...
95,what is the repercussion of the environment du...,The future of telehealth stays positive\nTo as...,1141,0.449606,513,14,0,0,0.0
96,impact of covid 19 pandemic on office space an...,The future of telehealth stays positive\nTo as...,1141,0.449606,513,14,0,0,0.0
97,contribution of handicrafts visual arts litera...,The future of telehealth stays positive\nTo as...,1141,0.449606,513,14,0,0,0.0
98,how covid 19 is impacting payment preferences,The future of telehealth stays positive\nTo as...,1141,0.449606,513,14,0,0,0.0


In [13]:
final = df.drop(columns=["corps"])  # Drop the "corps" column from the DataFrame
final

,title,total word count,percentage_complex_word,complex_word_count,AverageSentenceLenght,positive_score,negative_score,polarity_score
0,rising it cities and its impact on the economy...,1141,0.449606,513,14,0,0,0.0
1,rising it cities and their impact on the econo...,1141,0.449606,513,14,0,0,0.0
2,internet demands evolution communication impac...,1141,0.449606,513,14,0,0,0.0
3,rise of cybercrime and its effect in upcoming ...,1141,0.449606,513,14,0,0,0.0
4,ott platform and its impact on the entertainme...,1141,0.449606,513,14,0,0,0.0
...,...,...,...,...,...,...,...,...
95,what is the repercussion of the environment du...,1141,0.449606,513,14,0,0,0.0
96,impact of covid 19 pandemic on office space an...,1141,0.449606,513,14,0,0,0.0
97,contribution of handicrafts visual arts litera...,1141,0.449606,513,14,0,0,0.0
98,how covid 19 is impacting payment preferences,1141,0.449606,513,14,0,0,0.0


In [15]:
final.to_excel('Output Data Structure.xlsx')
